In [1]:
import Data_Simulator as ds

ds.sim_data

,GlobalDefectRate,AbsVolDeviation,AisleHoldPct,NumPickEvents,NumPickEventsClique,NumPicks,NumPicksClique,DefectInRelatedReceive,TimeInLocation,CurrentMaxVolume,Storage_M,Storage_S,Storage_XS,Defect
0,0.012363,3.261175,0.00,11,13,8,43,0,5.0,56.142236,True,False,False,0
1,0.006493,0.863345,0.00,1,21,27,44,0,6.0,19.121542,False,True,False,0
2,0.014716,1.547316,0.04,25,7,10,36,0,9.0,24.794522,False,True,False,0
3,0.009960,0.676915,0.00,7,1,4,40,0,2.0,37.773081,False,False,False,0
4,0.015507,3.047066,0.04,2,6,25,23,0,5.0,29.425188,True,False,False,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,0.000000,0.609795,0.00,4,1,34,12,0,8.0,28.353516,False,False,False,0
9996,0.021814,0.361120,0.00,7,2,0,22,0,4.0,11.061271,False,True,False,1
9997,0.009446,1.058161,0.06,1,15,17,19,0,9.0,21.732961,True,False,False,0
9998,0.015408,1.451685,0.04,5,7,8,39,0,5.0,26.959386,True,False,False,1
